#1. Install and Import Dependencies

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [3]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# load the pretrained model
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [4]:
# pass a string to tokenize our model
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [5]:
tokens[0]

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   102])

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [8]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [9]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [10]:
reviews

[]

# 5. Load Reviews into DataFrame and Score

In [11]:
import numpy as np
import pandas as pd

In [14]:
# Create a DataFrame 'df' from the array 'reviews' with a single column named 'review'
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
# Function to calculate sentiment score of a given review
def sentiment_score(review):
    # Tokenize the review and obtain tokens
    tokens = tokenizer.encode(review, return_tensors='pt')

    # Pass tokens through the model to get sentiment prediction
    result = model(tokens)

    # Get the index of the maximum value in the logits and convert to integer
    # Shift sentiment score from 0-based to 1-based and return
    return int(torch.argmax(result.logits)) + 1


In [ ]:
# Obtain sentiment score for the review at index 1 of the DataFrame 'df'
sentiment_score(df['review'].iloc[1])


In [ ]:
# Add a new column 'sentiment' to DataFrame 'df' by applying the sentiment_score function
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

In [ ]:
df['review'].iloc[3]

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) +1


In [ ]:
sentiment_score(df['review'].iloc[1])

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df['review']

end of project